### *Module Loading*

In [ ]:
import sys
import shutil
import cv2
from IPython import display

### *External Module Loading*

In [ ]:
external_modules_path = '..\\nn_likelihood_modules'
sys.path.append(external_modules_path)

In [ ]:
from common_imports import *
from common_use_functions import *
from constant import *

### *GPU verification*

In [ ]:
# Get the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nb_gpu = torch.cuda.device_count()
if nb_gpu > 0:
    print(torch.cuda.get_device_name(0))
else:
    print("CPU")

### *Working directory*

In [ ]:
# Current path
current_path = os.path.abspath(os.getcwd())

### *Load configurations and data*

In [ ]:
"""
All the parameters in this part should be configured
"""
# Experience path
experim_path = current_path

# File extensions
json_ext = '.json'
np_ext = '.npy'
csv_ext = '.csv'

# The tiny-imagenet folder
tiny_imagenet_path = path_join(experim_path, 'tiny-imagenet-200')
tiny_imagenet_val_path = path_join(tiny_imagenet_path, 'val')
tiny_imagenet_train_path = path_join(tiny_imagenet_path, 'train')
tiny_imagenet_reform_path = path_join(experim_path, 'tiny-imagenet-200-reform')
tiny_imagenet_reform_val_path = path_join(tiny_imagenet_reform_path, 'reform_val')
tiny_imagenet_reform_train_path = path_join(tiny_imagenet_reform_path, 'reform_train')

# The tiny-imagenet-c folder
tiny_imagenet_c_path = path_join(experim_path, 'Tiny-ImageNet-C')
tiny_imagenet_c_reform_path = path_join(experim_path, 'Tiny-ImageNet-C-reform')

# Resized image size
resized_image_size = 256

### *Tiny-imagenet reformulation*

In [ ]:
# Arrange the tiny imagenet validation set
tiny_imagenet_val_annotations = pd.read_csv(path_join(tiny_imagenet_val_path, 'val_annotations.txt'),
                                            names=['file', 'label_code', 'b_x0', 'b_y0', 'b_x1', 'b_y1'], header=None, sep='\t')

In [ ]:
# Create the folder that will contain the reformed images
create_directory(tiny_imagenet_reform_path)

In [ ]:
# Create the reformulated validation set folder
create_directory(tiny_imagenet_reform_val_path)
# Create the reformed validation set with the same structure as the train set
uniq_label_codes = tiny_imagenet_val_annotations['label_code'].unique()
for label_code in uniq_label_codes:
    create_directory(path_join(tiny_imagenet_reform_val_path, label_code))
# Copy the image to corresponding subfolder
for _, image_item in tqdm(tiny_imagenet_val_annotations.iterrows(), desc='Copied images', total=tiny_imagenet_val_annotations.shape[0]):
    ## Resize the image and then save
    # Read the image
    image = cv2.imread(path_join(tiny_imagenet_val_path, 'images', image_item['file']))
    # Resize the image
    resized_image = cv2.resize(image, (resized_image_size,resized_image_size))
    # Save the resized image     
    cv2.imwrite(path_join(tiny_imagenet_reform_val_path, image_item['label_code'], image_item['file']), resized_image) 
#     # Code to copy directly the images     
#     shutil.copy(path_join(tiny_imagenet_val_path, 'images', image_item['file']), path_join(tiny_imagenet_reform_val_path, image_item['label_code']))

In [ ]:
## Reform the training set
# Create the intial folder
create_directory(tiny_imagenet_reform_train_path)
# Copy the folder and images
for label_code in tqdm(contents_of_folder(tiny_imagenet_train_path), desc='Copied classes'):
    # Create the copied folder
    create_directory(path_join(tiny_imagenet_reform_train_path, label_code), display=False)
    # Get the image contents
    current_label_image_files = contents_of_folder(path_join(tiny_imagenet_train_path, label_code, 'images'))
    # Copy the images
    for image_file in current_label_image_files:
        ## Resize the image and then save
        # Read the image
        image = cv2.imread(path_join(tiny_imagenet_train_path, label_code, 'images', image_file))
        # Resize the image
        resized_image = cv2.resize(image, (resized_image_size,resized_image_size))
        # Save the resized image     
        cv2.imwrite(path_join(tiny_imagenet_reform_train_path, label_code, image_file), resized_image) 
#         # Code to copy directly the images  
#         shutil.copy(path_join(tiny_imagenet_train_path, label_code, 'images', image_file),
#                     path_join(tiny_imagenet_reform_train_path, label_code))

### *Tiny-imagenet-c reformulation*

In [ ]:
# Create the reformed dataset folder
create_directory(tiny_imagenet_c_reform_path)

In [ ]:
## Reform the images in the tiny-imagenet-c dataset
# Read the different type of modifications
transformations = contents_of_folder(tiny_imagenet_c_path)
# Iterate over the transformation types for the reformulation
for transformation in tqdm(transformations, desc='Copied transformations'):
    # Build the current transformation folder path
    current_transformation_path = path_join(tiny_imagenet_c_path, transformation)
    # Get all the severity levels
    severities = contents_of_folder(current_transformation_path)
    # Iterate over all the severity levels
    for severity in severities:
        # Build the current severity folder path
        current_severity_path = path_join(current_transformation_path, severity)
        # Create the reformulation folder
        current_severity_reform_path = path_join(tiny_imagenet_c_reform_path, transformation+'_'+severity)
        create_directory(current_severity_reform_path, display=False)
        # Get the label codes
        label_codes = contents_of_folder(current_severity_path)
        # Iterate over the label codes for resizing the images
        for label_code in label_codes:
            # Build the current label code folder path
            current_label_code_folder_path = path_join(current_severity_path, label_code)
            # Get the image contents
            current_label_image_files = contents_of_folder(current_label_code_folder_path)
            # Build the reformed label code image path
            current_label_code_reform_path = path_join(current_severity_reform_path, label_code)
            create_directory(current_label_code_reform_path, display=False)   
            # Copy the images
            for image_file in current_label_image_files:
                ## Resize the image and then save
                # Read the image
                image = cv2.imread(path_join(current_label_code_folder_path, image_file))
                # Resize the image
                resized_image = cv2.resize(image, (resized_image_size,resized_image_size))
                # Save the resized image     
                cv2.imwrite(path_join(current_label_code_reform_path, image_file), resized_image) 